# Relatório do senso das árvores de pinus de na cidade de Nova York em 2015

Esse conjunto de dados inclui um registro para cada árvore na cidade de Nova York e inclui a localização da árvore por bairro e latitude/longitude, espécies por nome latino e nomes comuns, tamanho e  saúde. O censo de árvores de 2015 foi conduzido pela equipe da 'NYC Parks and Recreation, TreesCount!', equipe do programa e centenas de voluntários.

**Datasets** 

[New York NTA](**Cr%C3%A9ditos**%20-%20%5Bhttps://data.cityofnewyork.us/City-Government/NTA-map/d3qk-pfyz%5D)

[Tree Census in New York City](https://www.kaggle.com/datasets/nycparks/tree-census)

# Resumo dos dados

In [ ]:
import pandas as pd
from json import load
import plotly.graph_objects as go


Importe os dados e visualize os nomes das colunas

In [ ]:
'''
#Processando dados originais
df_2015=pd.read_csv('new_york_tree_census_2015.csv')
df_2015[['spc_common','tree_dbh', 'stump_diam','status','health','problems','problems', 'root_stone','root_grate', 'root_other','nta','latitude','longitude']].head()
temp_df=df_2015.dropna().loc[df_2015.dropna()['spc_common'].str.contains('pine')][['spc_common','tree_dbh', 'stump_diam','status','health','problems','problems', 'root_stone','root_grate', 'root_other','nta','latitude','longitude']]
'''
temp_df = pd.read_csv('pine_NY_2015.csv')
temp_df.head(4)

Importando dado geográfico dos bairros.

In [ ]:
f = open('NTA_map.geojson')
nta_json = load(f)
for i in list(range(0,len(nta_json['features']))):
    json_id=nta_json['features'][i]['properties']['ntacode']
    nta_json['features'][i]['id']=json_id

# Funções de gráfico

In [ ]:
#plotlyTable
def PotlyTable (df):
    fig = go.Figure(data=[go.Table(header=dict(values=list(df.columns),
                                               fill_color='paleturquoise',
                                               align='center'),
                                   cells=dict(values=[df[col] for col in list(df.columns.values) if True],
                                              fill_color='lavender',
                                              align='center',
                                             height=30))])
    return fig

#PieChartPlotly
def PieChartPlotly (df,col):
    #df = Dataset
    #col = Coluna para agrupar os dados
    countCol = df.groupby(by=col).size()
    labels = list(countCol.index)
    values = list(countCol.values)
    figPie = go.Figure(data=[go.Pie(labels=labels,
                                    values=values,
                                    textinfo='label+percent+value',
                                    insidetextorientation='radial')
                            ])
    return figPie

#BoxPlotPlotly
def BoxPlotPlotly(df,col, colValue):
    data=[]
    for t in df[col].value_counts().index:
        data.append(go.Box(y=df[colValue].loc[df[col]==t],
                       name = t))
    BoxPlotfig = go.Figure(data=data)
    return BoxPlotfig

#ChoropletEspPlotly
def ChoropletEspPlotly(df,col_count,option,georef,colref,name):
    ChoropletGRaf = go.Choroplethmapbox(geojson=georef, 
                                locations=df[colref].loc[df[col_count]==option].value_counts().index, 
                                z=df[colref].loc[df[col_count]==option].value_counts().values,
                                colorbar=dict(title=name),
                                colorscale="YlGn"
                                )
    return ChoropletGRaf

#ScatterMapPotly
def ScatterMapPotly(df,option,colOption,label):
    Scatter = go.Scattermapbox(lat=df.loc[df[colOption]==option]['latitude'],
                           lon=df.loc[df[colOption]==option]['longitude'],
                           mode='markers',
                           marker=go.scattermapbox.Marker(size=5,color='black'),
                           hovertext=df.loc[df[colOption]==option][label])
    return Scatter

# Gráficos

In [ ]:
simpletable = temp_df.loc[:,['spc_common','tree_dbh','health','nta','latitude','longitude']].head(10) #Tabela resumida
PotlyTable(simpletable).update_layout(margin={"r":10,"t":50,"l":0,"b":0},title='Amostra do censo de Espécies de Pinus em NY').show()

In [ ]:
figPie = PieChartPlotly(temp_df,"spc_common").update_layout(margin={"r":0,"t":50,"l":0,"b":0}, title='Percentual de Espécies de Pinus em NY')
figPie.show()

In [ ]:
BoxPlotfig = BoxPlotPlotly(df = temp_df, col = 'spc_common',colValue = 'tree_dbh').update_layout(title="Box-plot DAP por Espécie",yaxis=dict( title='DAP'))
BoxPlotfig.show()

# Choroplet Geral

In [ ]:
data=[]
data.append(go.Choroplethmapbox(geojson=nta_json, 
                                locations=temp_df.nta.value_counts().index, 
                                z=temp_df.nta.value_counts().values,
                                colorbar=dict(title='Num. Árvores'),
                                colorscale="YlGn",marker_opacity=0.3
                                ))

data[0]['visible']=True


layout=go.Layout(mapbox_style="white-bg",
                 mapbox_layers=[{"below": 'traces',
                                 "sourcetype": "raster",
                                 "sourceattribution": "United States Geological Survey",
                                 "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]}],
                 mapbox_zoom=8.5, mapbox_center = {"lat": 40.7, "lon": -73.86})

    
fig = go.Figure(data=data,layout=layout)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Choroplet por especie

In [ ]:
scp = [t for t in temp_df['spc_common'].value_counts().index if True]
scp

In [ ]:
x = 1
ChoropletGraf = go.Figure(ChoropletEspPlotly(df = temp_df,
                                   col_count = 'spc_common',
                                   option = scp[x],
                                   georef =nta_json,
                                   colref='nta',
                                   name='Num. Árvores')).update_layout(margin={"r":0,"t":50,"l":0,"b":0},
                                                                      mapbox_style="carto-positron",
                                                                      mapbox_zoom=8.5,
                                                                      mapbox_center = {"lat": 40.7, "lon": -73.86},title=f"Número de árvores de {scp[x]} por Bairro de NY")
ChoropletGraf.show()

# Histogram especie

In [ ]:
x = 0
BoxPlotfig = BoxPlotPlotly(df = temp_df.loc[temp_df['spc_common']==scp[x]],
                           col = 'spc_common',
                           colValue = 'tree_dbh').update_layout(title="Box-plot DAP por Espécie",yaxis=dict( title='DAP'))
BoxPlotfig.show()

# Pontos de Pinus

Mapa de pontos dos tipos de pinheiros. O mapa é gerado para corresponder a todos ou usos parciais da string de entrada. Cada correspondência de espécies recebe uma cor e permite que as espécies sejam selecionadas ou desmarcadas na legenda do lado direito.

In [ ]:
Scatter = ScatterMapPotly(df=temp_df,
                          option=scp[x],
                          colOption='spc_common',
                          label='nta')



Choroplet = ChoropletEspPlotly(df = temp_df,
                                   col_count = 'spc_common',
                                   option = scp[x],
                                   georef =nta_json,
                                   colref='nta',
                                   name='Num. Árvores')

layout=go.Layout(mapbox_style="carto-positron",
                 mapbox_zoom=9,
                 mapbox_center = {"lat": 40.7, "lon": -73.86},
                 margin={"r":0,"t":10,"l":0,"b":0})

fig = go.Figure(data=[Choroplet,Scatter],layout=layout)
fig.show()